In [13]:

import pandas as pd
import numpy as np
df = pd.read_csv("training_data.csv")

# Map labels to integers if needed
df['increase_stock_label'] = df['increase_stock'].map({
    "low_bike_demand": 0,
    "high_bike_demand": 1
})

df["month"] = df["month"] -1

y = df['increase_stock_label'] 
X = df.drop(columns=["increase_stock", "increase_stock_label"])

X["hour_sin"] = np.sin(2 * np.pi * X["hour_of_day"] / 24)
X["hour_cos"] = np.cos(2 * np.pi * X["hour_of_day"] / 24)

X["day_sin"] = np.sin(2 * np.pi * X["day_of_week"] / 7)
X["day_cos"] = np.cos(2 * np.pi * X["day_of_week"] / 7)

X["month_sin"] = np.sin(2 * np.pi * X["month"] / 12)
X["month_cos"] = np.cos(2 * np.pi * X["month"] / 12)

X = X.drop(columns=["hour_of_day", "day_of_week", "month"])

np.random.seed(3)
N = len(X)
M = np.ceil(0.8 * N).astype(int)  # Number of training data

idx = np.random.permutation(N)

X_train, X_test = X.iloc[idx[:M]], X.iloc[idx[M:]]
y_train, y_test = y.iloc[idx[:M]], y.iloc[idx[M:]]




In [14]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from load_data import X_train, X_test, y_test, y_train

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Check shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Normalize input (very important!)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define model
model = keras.Sequential([
    layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid")  # Binary classification
])

# Compile model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Train
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)


X_train shape: (1280, 15)
y_train shape: (1280,)
Epoch 1/50


/Users/tlindberg/Desktop/smask/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8188 - loss: 0.4883 - val_accuracy: 0.8281 - val_loss: 0.3891
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8497 - loss: 0.3266 - val_accuracy: 0.8242 - val_loss: 0.3757
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8613 - loss: 0.3029 - val_accuracy: 0.8359 - val_loss: 0.3688
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - accuracy: 0.8660 - loss: 0.2898 - val_accuracy: 0.8438 - val_loss: 0.3633
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - accuracy: 0.8769 - loss: 0.2525 - val_accuracy: 0.8555 - val_loss: 0.3464
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.9019 - loss: 0.2343 - val_accuracy: 0.8516 - val_loss: 0.3457
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - accuracy: 0.8964 - loss: 0.2421 - val_accuracy: 0.8516 - val_loss: 0.3437
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9169 - loss: 0.2184 - val_accuracy: 0.8633 - val_loss: 0.

In [15]:
X_train.shape

(1280, 15)

In [16]:
X_test.shape

(320, 15)

In [17]:


predict = [0 if x < 0.5 else 1 for x in model.predict(X_test)]



prediction = predict == y_test

sum(prediction) / len(prediction)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step


0.86875

In [18]:
len(predict)

320